In [1]:
import pwn
import json
from tqdm import tqdm
import random

r = pwn.process("./run_chall.sh")
r.recvuntil("p = ")
p = int(r.recvline())
r.recvuntil("Here is the encrypted flag:")
r.recvline()
flag = r.recvline()
print(flag)
flag = json.loads(flag)

def encrypt(inp):
    r.recvuntil("What would you like to encrypt?")
    r.sendline(json.dumps(inp))
    return json.loads(r.recvline())

F = GF(p)
dims = 50
num_samples = 60
outs = []
inps = []

for i in tqdm(range(num_samples)):
    inp = [random.randint(-1,2) for _ in range(dims)]
    outs.append(encrypt(inp))
    inps.append([F(x) for x in inp])

outs = matrix(ZZ,outs).T

M = block_matrix(2,1,[[identity_matrix(ZZ,len(outs[0]))*p],[outs]])
lll = M.LLL()

# This matrix is almost the correct second layer weights matrix,
# Up to permutation, sign and a couple wrong elements
T = Matrix(F,outs[:,:dims]).T.inverse() * Matrix(F,lll[dims:dims*2, :dims]).T 

e = (p+1)//2

def out_to_mid(o):
    return [F(x)^e for x in (o * T)]

mids = []
for i in range(num_samples):
    mids.append(out_to_mid(outs[:,i].T[0]))

# This is almost the correct first layer weights matrix
L1 = (Matrix(F,inps[:dims]).inverse() * Matrix(F,mids[:dims])).T

good_dims = set(range(dims))
for i in range(num_samples):
    mid_recon = L1 * vector(F,inps[i])
    for j in range(len(mid_recon)):
        if mids[i][j] != mid_recon[j] and j in good_dims:
            good_dims.remove(j)
print("Number correct dimensions:",len(good_dims))

L1_good = L1[list(good_dims)]
L1_good = Matrix(ZZ,[[int(x) if int(x)<p//2 else int(x)-p for x in row] for row in L1_good]) # Need to translage p-x to -x
    
def small_solution(M, b):
    # M*a == b for a small

    b = Matrix(b)
    n = M.ncols()
    m = M.nrows()

    B = block_matrix(2,2,[[b*10000,Matrix([[0]*n])],[M.T*10000,identity_matrix(n)]])

    lll = B.LLL()
    return (list(lll[0,m:])[0])
    
def decode(out_v):
    mid_good = Matrix(F,out_to_mid(out_v))[0,list(good_dims)]
    mid_good = Matrix(ZZ,[[int(x) if int(x)<p//2 else int(x)-p for x in row] for row in mid_good])

    return small_solution(L1_good, mid_good)

dec_flag = decode(vector(F,flag))

print(bytes([abs(x) for x in dec_flag]))

[x] Starting local process './run_chall.sh'
[+] Starting local process './run_chall.sh': pid 9751


 15%|█▌        | 9/60 [00:00<00:00, 84.85it/s]

b'[96497587833895178897146585081118619750, 19155983810463007923205098412389125908, 77403194945693884535551646882477900709, 71963690676437430092729130428434320105, 109974316959094934321658498652227825839, 63672052454251891160784353093469933358, 70320629393567473275290253471172573151, 3709456898888061492713311792754486779, 46205106068716053646737616084061000963, 15267548984556554821603931206454110826, 18212072540755338734102041294159887064, 66889565110886339601420266006298072587, 64372897668280347435536775442761279492, 2318855990639540102842855013461196598, 54575739041489529624788304593578480667, 37651745214517173948608278526297938602, 81891757430644587226352040520303634064, 114396359576097918103775937256150622957, 70164545923509706580750505144276375993, 67994096358301452403055774907787765190, 45151548674659747224425388597214518056, 73849214083234226939726640602802606163, 108214806200773685887979631886176176059, 65810134552462715336484479422578089629, 723424849281115647658070042247023247

100%|██████████| 60/60 [00:00<00:00, 133.94it/s]


Number correct dimensions: 39
b'flag{aaaaaaaaaaaaaaaaaasdlkjfaaalsdkjfsaasdsd;lfs}'
